# 랭체인의 구성 요소

## 모델과 프롬프트

In [1]:
from langchain_openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
llm = OpenAI(openai_api_key=openai_api_key)
print(llm.invoke('농담을 해봐.'))



왜 곰은 냉장고를 열지 않을까? 
- 냉장고 안에 살던 냉동곰을 무서워해서!


In [3]:
from langchain import PromptTemplate

template = """문장: {sentence}
{language}로 번역:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

print(prompt.format(sentence = "탁자 위에 고양이가 있다", language = "영어"))

문장: 탁자 위에 고양이가 있다
영어로 번역:


## 데이터 연결

### 문서 로더

In [4]:
import csv

# Sample data
data = [
    ['이름', '나이', '도시'],
    ['존', 25, '뉴욕'],
    ['에밀리', 28, '로스엔젤레스'],
    ['미카엘', 22, '시카고']
]

# File name
file_name = 'sample.csv'

# 데이터를 CSV 파일에 기록
with open(file_name, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(data)

print(f'예제 CSV 파일 "{file_name}"를 만들었습니다.')



예제 CSV 파일 "sample.csv"를 만들었습니다.


In [5]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='sample.csv')
data = loader.load()
print(data)

[Document(metadata={'source': 'sample.csv', 'row': 0}, page_content='이름: 존\n나이: 25\n도시: 뉴욕'), Document(metadata={'source': 'sample.csv', 'row': 1}, page_content='이름: 에밀리\n나이: 28\n도시: 로스엔젤레스'), Document(metadata={'source': 'sample.csv', 'row': 2}, page_content='이름: 미카엘\n나이: 22\n도시: 시카고')]


### 문서 분할기

In [6]:
# 산과 자연에 대한 샘플 문장
content = """고요한 풍경 속에서 우뚝 솟은 산들은 자연의 아름다움을 지키는 장엄한 수호자처럼 서 있습니다.
청량한 산 공기는 고요함의 속삭임을 전해주며, 바스락거리는 잎사귀들은 야생의 교향곡을 작곡합니다.
자연의 팔레트는 산을 초록과 갈색의 색조로 칠해 경이로운 광경을 만들어냅니다.
해가 뜨면, 황금빛 광채가 산 정상에 비치며, 손길 닿지 않은 야생의 세계를 비춥니다."""

# 파일명
file_name = 'mountain.txt'

# 텍스트 파일에 내용 쓰기
with open(file_name, 'w') as txtfile:
    txtfile.write(content)

# 샘플 텍스트 파일 "mountain.txt" 생성 및 저장.
# print(f'샘플 텍스트 파일 "{file_name}"를 만들었습니다.')

with open('mountain.txt') as f:
    mountain = f.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len
)

texts = text_splitter.create_documents([mountain])
print(texts[0])
print(texts[1])
print(texts[2])


page_content='고요한 풍경 속에서 우뚝 솟은 산들은 자연의 아름다움을 지키는 장엄한 수호자처럼 서 있습니다.'
page_content='청량한 산 공기는 고요함의 속삭임을 전해주며, 바스락거리는 잎사귀들은 야생의 교향곡을 작곡합니다.
자연의 팔레트는 산을 초록과 갈색의 색조로 칠해 경이로운 광경을 만들어냅니다.'
page_content='해가 뜨면, 황금빛 광채가 산 정상에 비치며, 손길 닿지 않은 야생의 세계를 비춥니다.'


### 텍스트 임베딩 모델

In [7]:
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]

embeddings_model = OpenAIEmbeddings(model ='text-embedding-3-small' )

embeddings = embeddings_model.embed_documents(
    [
        "Good morning!",
        "Oh, hello!",
        "I want to report an accident",
        "Sorry to hear that. May I ask your name?",
        "Sure, Mario Rossi."
    ]
)

print("임베드된 문서:")
print(f"Number of vector: {len(embeddings)}; Dimension of each vector: {len(embeddings[0])}")

embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")

print("임베드 질의:")
print(f"Dimension of the vector: {len(embedded_query)}")
print(f"Sample of the first 5 elements of the vector: {embedded_query[:5]}")


임베드된 문서:
Number of vector: 5; Dimension of each vector: 1536
임베드 질의:
Dimension of the vector: 1536
Sample of the first 5 elements of the vector: [-0.010634176433086395, -0.01016946416348219, -0.0020040736999362707, 0.023065242916345596, -0.026829415932297707]


In [8]:
# 대화를 txt 파일에 저장
# 대화 행 목록
dialogue_lines = [
    "Good morning!",
    "Oh, hello!",
    "I want to report an accident",
    "Sorry to hear that. May I ask your name?",
    "Sure, Mario Rossi."
]

# 파일명
file_name = 'dialogue.txt'

# 대화 행들을 텍스트 파일에 기록
with open(file_name, 'w') as txtfile:
    for line in dialogue_lines:
        txtfile.write(line + '\n')

print(f'대화 텍스트 파일 "{file_name}"를 만들었습니다.')


대화 텍스트 파일 "dialogue.txt"를 만들었습니다.


### 벡터 스토어

In [9]:
from langchain.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]

# 문서를 로드해 청크들로 분할하고, 각 청크를 임베드해 벡터 스토어에 로드

raw_documents = TextLoader('dialogue.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0, separator = "\n",)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())


c:\Users\yong\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [10]:
query = "What is the reason for calling?"
docs = db.similarity_search(query)
print(docs[0].page_content)

I want to report an accident


In [11]:
print(documents[2])

page_content='Sorry to hear that. May I ask your name?' metadata={'source': 'dialogue.txt'}


### 검색기

In [12]:
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI

retriever = db.as_retriever()

In [13]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

query = "What was the reason of the call?"
output = qa.invoke(query)
output['result']

' The reason for the call was to report an accident.'

## 메모리

In [14]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import OpenAI

memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input": "안녕하세요, AI에 관한 에세이를 쓸 아이디어를 찾고 있어요"}, {"output": "안녕하세요, LLM에 관해 써보면 어떨까요?"})

memory.load_memory_variables({})

{'history': '\nThe human is looking for ideas to write an essay about AI. The AI suggests writing about LLM.'}

In [15]:
ConversationSummaryMemory.save_context?

Signature:
ConversationSummaryMemory.save_context(
    self,
    inputs: 'Dict[str, Any]',
    outputs: 'Dict[str, str]',
) -> 'None'
Docstring: Save context from this conversation to buffer.
File:      c:\users\yong\appdata\local\programs\python\python311\lib\site-packages\langchain\memory\summary.py
Type:      function

## 체인

### 단순 체인

In [16]:
from langchain_openai import OpenAI
from langchain import PromptTemplate, LLMChain

template = """문장: {sentence}
{language}로 번역:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

llm = OpenAI(temperature=0)

llm_chain = LLMChain(prompt=prompt, llm=llm)

llm_chain.predict(sentence="탁자 위에 고양이가 있어요", language="영어")

c:\Users\yong\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


' There is a cat on the table.'

In [17]:
# LLMChain deprecation 해결

from langchain_openai import OpenAI
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnablePassthrough

template = """문장: {sentence}
{language}로 번역:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

llm = OpenAI(temperature=0)

output_parser = StrOutputParser()

chain = RunnableSequence(
    {
        "sentence": RunnablePassthrough(),
        "language": RunnablePassthrough()
    }
    | prompt
    | llm
    | output_parser
)

result = chain.invoke({
    "sentence": "탁자 위에 고양이가 있어요",
    "language": "영어"
})
print(result)



There is a cat on the table.


### 라우터 체인

In [18]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

llm = OpenAI()

itinerary_template = """당신은 휴가 일정 조수입니다. \
당신은 고객이 최고의 목적지와 일정을 찾도록 도와줍니다. \
당신은 고객의 선호에 따라 최적화된 일정을 작성하는 데 도움을 줍니다.

여기에 질문이 있습니다:
{input}"""

restaurant_template = """당신은 레스토랑 예약 조수입니다. \
당신은 고객의 손님 수와 음식 선호를 확인합니다. \
특별한 조건을 고려해야 하는지 주의합니다.

여기에 질문이 있습니다:
{input}"""

prompt_infos = [
    {
        "name": "여행 일정",
        "description": "여행 일정 작성을 돕습니다",
        "prompt_template": itinerary_template,
    },
    {
        "name": "레스토랑",
        "description": "고객의 레스토랑 예약을 도와줍니다",
        "prompt_template": restaurant_template,
    },
]

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)



c:\Users\yong\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(
c:\Users\yong\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  warn_deprecated(


In [19]:
print(chain.invoke("밀라노에서 베니스까지 자동차로 여행하려고 합니다. 중간에 들릴 만한 명소가 있나요?."))



> Entering new MultiPromptChain chain...
여행 일정: {'input': '밀라노에서 베니스까지 자동차로 여행하려고 합니다. 중간에 들릴 만한 명소가 있나요?.'}
> Finished chain.
{'input': '밀라노에서 베니스까지 자동차로 여행하려고 합니다. 중간에 들릴 만한 명소가 있나요?.', 'text': '\n\n1. 밀라노에서 베니스까지의 여행 시간은 얼마나 걸리나요?\n2. 자동차 여행으로 가는 것보다 기차나 비행기로 가는 것이 더 빠른가요?\n3. 중간에 들릴 만한 명소는 어떤 곳이 있나요? 역사적인 곳이나 볼거리가 있는 곳 등\n4. 자동차 여행으로 가는데 필요한 경비는 얼마나 되나요?\n5. 여행 중에 주차할 수 있는 곳이 어디인가요?\n6. 베니스에서 머물기 좋은 호텔이나 숙박 시설은 어디인가요?\n7. 베니스에서 즐길 수 있는 여행 액티비티는 무엇이 있나요?\n8. 베니스에서 밀라노'}


In [20]:
print(chain.invoke("오늘 저녁 식사를 예약하고 싶어요"))



> Entering new MultiPromptChain chain...
레스토랑: {'input': '오늘 저녁 식사를 예약하고 싶어요'}
> Finished chain.
{'input': '오늘 저녁 식사를 예약하고 싶어요', 'text': '. 몇 명이서 오실 건가요? 몇 명의 성인과 어린이인지 알려주세요.\n어떤 종류의 음식을 선호하시나요? 고기, 해산물, 채식 등 다양한 옵션 중에서 선택할 수 있습니다.\n알레르기나 식이 요구 사항이 있으신가요? 예를 들어, 글루텐 또는 유제품 무첨가 요리를 원하시나요? 특별한 식이 요구 사항이 있다면 말씀해주세요.\n어떤 특별한 행사를 기념하시나요? 생일, 결혼 기념일 등 특별한 날에 맞춰 서비스를 제공할 수 있습니다.\n결제는 어떻게 하실 건가요? 현금, 카드,'}


### 시퀀셜 체인

In [21]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# 이 코드는 연극의 제목을 주면 시놉시스를 작성하는 LLMChain입니다.
llm = OpenAI(temperature=0)
template = """당신은 코미디언입니다. {topic}에 관한 농담을 생성하세요.
농담:"""
prompt_template = PromptTemplate(input_variables=["topic"], template=template)
joke_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """당신은 번역가입니다. 주어진 텍스트 입력을 {language}로 번역하세요.
번역:"""
prompt_template = PromptTemplate(input_variables=["language"], template=template)
translator_chain = LLMChain(llm=llm, prompt=prompt_template)

In [22]:
# 이 코드는 두 체인을 순서대로 실행하는 전체 체인입니다.
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[joke_chain, translator_chain], verbose=True)
translated_joke = overall_chain.run("고양이와 개")

c:\Users\yong\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
 "고양이와 개가 싸우면 누가 이길까요? 당연히 고양이죠, 개는 꼬리를 쫓는데 바쁘니까요!"
 If cats and dogs fight, who do you think will win? Of course, it's the cat, because the dog is too busy chasing its tail!

> Finished chain.


### 변환 체인

In [23]:
# string 모듈을 임포트합니다
import string

# 함수를 정의합니다
def rename_cat(inputs: dict) -> dict:
    # 파일을 읽기 모드로 엽니다
    text = inputs["text"]
    # 'cat'을 'Silvester the Cat'으로 바꿉니다
    new_text = text.replace('cat', 'Silvester the Cat')
    # 변경된 텍스트를 반환합니다
    return {"output_text": new_text}


In [24]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

with open("Cats&Dogs.txt") as f:
    cats_and_dogs = f.read()


import string



transform_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=rename_cat
)

template = """이 텍스트를 요약하세요:

{output_text}

요약:"""
prompt = PromptTemplate(input_variables=["output_text"], template=template)
llm_chain = LLMChain(llm=OpenAI(), prompt=prompt)

sequential_chain = SimpleSequentialChain(chains=[transform_chain, llm_chain])

sequential_chain.invoke(cats_and_dogs)

{'input': "\nThe Cat and the Dog\n\nThere was once a cat and a dog who lived in the same house. They did not get along very well, as they often fought over food, toys, and attention. The cat was clever and cunning, while the dog was loyal and friendly. The cat liked to tease the dog, and the dog liked to chase the cat.\n\nOne day, the cat decided to play a prank on the dog. He found a ball of yarn and tied it around the dog's tail. Then he hid behind a sofa and waited for the dog to notice. When the dog saw the yarn, he thought it was a toy and started to play with it. He ran around the house, trying to catch the yarn with his mouth. But every time he got close, the yarn moved away from him. The cat laughed silently as he watched the dog's futile attempts.\n\nThe dog soon realized that something was wrong. He looked behind him and saw that the yarn was attached to his tail. He tried to pull it off, but it was too tight. He felt angry and embarrassed. He wondered who did this to him. He

## 에이전트

In [30]:
from langchain import SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain_openai import OpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

import os
from dotenv import load_dotenv

load_dotenv(override=True)

os.environ["SERPAPI_API_KEY"]

search = SerpAPIWrapper()

In [31]:
tools = [Tool.from_function(
        func=search.run,
        name="Search",
        description="현재 이벤트에 관해 질문할 때 유용함"
    )]

agent = initialize_agent(tools, llm = OpenAI(), agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.invoke("아바타 2 개봉일은?")



> Entering new AgentExecutor chain...
 첫 번째로 어떤 검색어를 입력해야 할지 고민해볼 수 있습니다.
Action: Search
Action Input: "Avatar 2 release date"
Observation: December 16, 2022
Thought: 이 정보는 신뢰할만한 출처에서 가져왔는지 확인해볼 필요가 있습니다.
Action: Search
Action Input: "Avatar 2 release date official"
Observation: December 16, 2022
Thought: 확인 결과, 이 정보는 영화 제작사인 20세기 폭스의 공식 홈페이지에서 확인할 수 있었습니다.
Final Answer: December 16, 2022

> Finished chain.


{'input': '아바타 2 개봉일은?', 'output': 'December 16, 2022'}

# 허깅페이스 허브를 통해 LLM 사용하기

In [ ]:
#필요한 패키지 설치
!pip install python-dotenv   
!pip install langchain-huggingface


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/43.7 kB ? eta -:--:--
     ---------------------------------------- 43.7/43.7 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/417.5 kB ? eta -:--:--
   ---------------------------------------- 417.5/417.5 kB 8.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/227.1 kB ? eta -:--:--
   ---------------------------------------- 227.1/227.1 kB 7.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------- ----------------------- 0.9/2.2 MB 19.2 MB/s eta 0:00:01
   ------------------------------------ --- 2.0/2.2 MB 21.5 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 20.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---- ----------------------------------- 1.1/9.5 MB 36.0 MB/s eta 0:00:01
   ---------- ----------------------------- 2.6/9.5 MB 33.1 MB/s eta 0:00:01
   ---------------- ---


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
#옵션 1: .env 파일에서 토큰 가져기기

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["HUGGINGFACEHUB_API_TOKEN"][:3]  # 앞부분만 출력


'hf_'

In [ ]:
#옵션 2: getpass 함수를 사용해 토큰 가져오기

# from getpass import getpass

# HUGGINGFACEHUB_API_TOKEN = getpass()
# HUGGINGFACEHUB_API_TOKEN

In [33]:
from langchain import PromptTemplate, LLMChain
from langchain_huggingface import HuggingFaceEndpoint
question = "What was the first Disney movie?"

template = """Question: {question}

Answer: give a direct answer"""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [35]:
repo_id = "tiiuae/falcon-7b-instruct"  
llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_length=1000,
    temperature=0.5,
)
print(llm.invoke("what was the first disney movie?"))

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\yong\.cache\huggingface\token
Login successful

The first Disney movie was 'Snow White and the Seven Dwarfs' released in 1937.
